In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import sys
sys.path.append("..")

%matplotlib inline

import numpy as np

from sklearn.metrics import classification_report

import keras
from keras import metrics
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Flatten
from keras.models import Model, load_model
import keras.backend as k
from matplotlib import pyplot as plt
from IPython.display import clear_output

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout

from art.config import ART_DATA_PATH
# from art.attacks.evasion import PixelAttack
from art.estimators.classification import KerasClassifier
from art.utils import to_categorical, load_dataset, get_file

import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from algorithms.pixel_attack import PixelAttack

import logging

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter("[%(levelname)s] %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

In [3]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("cifar10"))
x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:500], y_test[:500]

In [4]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train.shape[1:]))
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=METRICS)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        1

In [5]:
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=15, batch_size=128, verbose=0)

[INFO] Inferred 17 hidden layers on Keras classifier.


In [6]:
preds = np.argmax(classifier.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
base_results = classifier.model.evaluate(x_test, y_test, verbose=1)
print("Classifier results before attack: ")
dict(zip(classifier.model.metrics_names, base_results))

Classifier results before attack: 


{'loss': 1.214788031578064,
 'tp': 239.0,
 'fp': 108.0,
 'tn': 4392.0,
 'fn': 261.0,
 'categorical_accuracy': 0.594,
 'precision': 0.6887608,
 'recall': 0.478,
 'auc': 0.91918266}

In [7]:
logger.info("Create PixelAttack attack")
adv_crafter = PixelAttack(classifier)
#logger.info("Craft attack on training examples")
#x_train_adv = adv_crafter.generate(x_train)
logger.info("Craft attack test examples")
x_test_adv = adv_crafter.generate(x_test)

[INFO] Create PixelAttack attack
[INFO] Craft attack test examples
[INFO] Attacking with minimal perturbation.
[INFO] Success rate of Attack: 18.80%


In [8]:
preds = np.argmax(classifier.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
adv_results = classifier.model.evaluate(x_test_adv, y_test, verbose=1)
print("Classifier results after attack: ")
dict(zip(classifier.model.metrics_names, adv_results))

Classifier results before attack: 


{'loss': 10.1599345703125,
 'tp': 179.0,
 'fp': 321.0,
 'tn': 4179.0,
 'fn': 321.0,
 'categorical_accuracy': 0.358,
 'precision': 0.358,
 'recall': 0.358,
 'auc': 0.6467477}

In [5]:
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=15, batch_size=128, verbose=0)

[INFO] Inferred 17 hidden layers on Keras classifier.


In [6]:
preds = np.argmax(classifier.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
base_results = classifier.model.evaluate(x_test, y_test, verbose=1)
print("Classifier results before attack: ")
dict(zip(classifier.model.metrics_names, base_results))

Classifier results before attack: 


{'loss': 1.275928005695343,
 'tp': 226.0,
 'fp': 124.0,
 'tn': 4376.0,
 'fn': 274.0,
 'categorical_accuracy': 0.536,
 'precision': 0.6457143,
 'recall': 0.452,
 'auc': 0.913084}

In [7]:
logger.info("Create PixelAttack attack")
adv_crafter = PixelAttack(classifier)
#logger.info("Craft attack on training examples")
#x_train_adv = adv_crafter.generate(x_train)
logger.info("Craft attack test examples")
x_test_adv = adv_crafter.generate(x_test)

[INFO] Create PixelAttack attack
[INFO] Craft attack test examples
[INFO] Attacking with minimal perturbation.
[INFO] Success rate of Attack: 19.80%


In [8]:
preds = np.argmax(classifier.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
adv_results = classifier.model.evaluate(x_test_adv, y_test, verbose=1)
print("Classifier results after attack: ")
dict(zip(classifier.model.metrics_names, adv_results))

Classifier results before attack: 


{'loss': 8.856630432128906,
 'tp': 218.0,
 'fp': 282.0,
 'tn': 4218.0,
 'fn': 282.0,
 'categorical_accuracy': 0.436,
 'precision': 0.436,
 'recall': 0.436,
 'auc': 0.6901522}